# **Opening a New Business in San Antonio, TX Informed by Foursquare Data**

## **D. Risius**  
### *1/1/2020*

**Introduction:** San Antonio, Texas is one of the fastest growing cities in the United States.  According to the [United States Census Bureau](https://www.census.gov/newsroom/press-releases/2018/estimates-cities.html), San Antonio topped the list of the fastest growing metro areas for 2017.  In previous analysis, we clustered and segmented neighborhoods in [Toronto](https://github.com/risiud/Coursera_Capstone/blob/master/Clustering%20Toronto%20Neighborhoods.ipynb) and New York city based on FourSquare venue data.  San Antonio is a very different city than either New York or Toronto.  For one, it is a very large city with relatively sparse population compared to the other cities.  According to [Wikopedia](https://en.wikipedia.org/wiki/San_Antonio), San Antonio city consists of around 1.5 million people within a land area of 461 square miles compared to 8.5 million for 303 square miles in [New York City](https://en.wikipedia.org/wiki/New_York_City) and 2.7 million for 243 square miles in [Toronto](https://en.wikipedia.org/wiki/Toronto).  The ethnicity of the three cities is also different.  San Antonio has a large hispanic influence with around 63% of residents of hispanic or latino origin.  New York is around 28% hispanic while Toronto is around 4% hispanic with a much larger proportion of Asian (40%) and European (48%) than San Antonio or New York.  This data may help us determine which types of venues may be more successful in different neighborhoods.

**Problem Statement:** Given our previous analysis clustering and segmenting neighborhoods using FourSquare data in New York City and Toronto, how does San Antonio, Texas compare in terms of most popular types of veneus?  If we wanted to open a new business in San Antonio, can we use the FourSquare data for the different clusters to inform a decision on what type of business to open and the best location around the city to start it?

**Data:** New York City and Toronto have well defined neighborhoods that helped us cluster the data.  San Antonio has some established neighborhoods, however many of the areas within the city are not defined within a particular neighborhood.  Therefore we can't use the same approach as we did with New York and Toronto as we would omit large portions of the city.  San Antonio consists of 87 seperate zip codes.  For analyzing San Antonio we will these zip codes intead and will map and cluster those using the geographical center of the zip code.  To get the geographic coordinates we used the website [San Antonio AreaConnect](https://sanantonio.areaconnect.com/zip2.htm?city=San%20Antonio&search=zip) which provides latitude/longitude coordinates for the various zip codes around San Antonio.  We will cluster these zip codes using the Foursquare location data similiar to the analysis in New York and Toronto.  Based on the cluster analysis, we will recommend ideas for new business venue(s) in particular zip codes.  The excerpt of the San Antonio zip code data below shows the data we will use for our analysis.     

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
#geocode(query, country_codes=None, exactly_one=True, timeout=DEFAULT_SENTINEL)
#geocode('78251', country_codes='US', exactly_one=True, timeout=DEFAULT_SENTINEL)

address = '78251'

geolocator = Nominatim(user_agent="SA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Zip Code 78251 are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Zip Code 78251 are 29.4361366, -98.683798853038.


In [3]:
url = "https://raw.githubusercontent.com/risiud/Coursera_Capstone/master/SanAntonioZips.csv"

neighborhoods = pd.read_csv(url)
neighborhoods.head()

,Zipcode,City,State,AreaCode,County,Latitude,Longitude
0,78201,San Antonio,TX,210,Bexar,29.472,-98.537
1,78202,San Antonio,TX,210,Bexar,29.422,-98.466
2,78203,San Antonio,TX,210,Bexar,29.415,-98.462
3,78204,San Antonio,TX,210,Bexar,29.397,-98.500
4,78205,San Antonio,TX,210,Bexar,29.424,-98.487


In [4]:
print('The dataframe has {} Zip Codes.'.format(
        len(neighborhoods['Zipcode'].unique())
    )
)

The dataframe has 87 Zip Codes.


And and map below shows the geographic position for each of the 87 zipcodes around San Antonio ![San Antonio Zip Codes](https://github.com/risiud/Coursera_Capstone/blob/master/SanAntonioNeighborhoods.JPG?raw=true)

In [5]:
address = 'San Antonio, TX'

geolocator = Nominatim(user_agent="SA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Antonio are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Antonio are 29.4246002, -98.4951405.


In [6]:
# create map of New York using latitude and longitude values
map_SA = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, zipcode in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Zipcode']):
    label = '{}'.format(zipcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_SA)  
    
map_SA

In [8]:
# Define Four-Square Credentials

CLIENT_ID = '2HVAO4WMSHFLNNF1LGIQJBZYIVPSF3WBJPPDORAVO1M5NSHN' # your Foursquare ID
CLIENT_SECRET = 'YXKMVUBPMFSNUKICEK0QAEIT0R0HK3GXS1FP5CY2NAQ12SNT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2HVAO4WMSHFLNNF1LGIQJBZYIVPSF3WBJPPDORAVO1M5NSHN
CLIENT_SECRET:YXKMVUBPMFSNUKICEK0QAEIT0R0HK3GXS1FP5CY2NAQ12SNT


In [37]:
# Lets explore zip code 78251
neighborhoods.loc[46, 'Zipcode']


78247

In [38]:
# Get the latitude and longitude values of zip 78251

neighborhood_latitude = neighborhoods.loc[46, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[46, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[46, 'Zipcode'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of 78247 are 29.588, -98.402.


In [39]:
# Now lets get the top 100 venues in zip 78251 within a 2500m radius

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=2HVAO4WMSHFLNNF1LGIQJBZYIVPSF3WBJPPDORAVO1M5NSHN&client_secret=YXKMVUBPMFSNUKICEK0QAEIT0R0HK3GXS1FP5CY2NAQ12SNT&v=20180605&ll=29.588,-98.402&radius=2500&limit=100'

In [40]:
# Send the get requests and examine the results

results = requests.get(url).json()
#results

In [41]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [42]:
# Now clean the data and put it in a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Heb,Market,29.595079,-98.417521
1,True Texas BBQ,BBQ Joint,29.595557,-98.417070
2,Wu 's New York style authentic Chinese food,Chinese Restaurant,29.588250,-98.378817
3,Lefty's,Bar,29.588089,-98.378762
4,LA Fitness,Gym / Fitness Center,29.597768,-98.421240
5,Valero Corner Store,Convenience Store,29.578937,-98.392189
6,Milbergers Nursery,Miscellaneous Shop,29.600086,-98.416342
7,Gold's Gym,Gym / Fitness Center,29.604788,-98.416637
8,Los Arcos Mexican Grill,Mexican Restaurant,29.588201,-98.378802
9,Taco Bell,Fast Food Restaurant,29.603425,-98.418374


In [15]:
#Lets get the top 100 venues in each neighborhood

def getNearbyVenues(names, latitudes, longitudes, radius=2500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
SA_venues = getNearbyVenues(names=neighborhoods['Zipcode'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

78201
78202
78203
78204
78205
78206
78207
78208
78209
78210
78211
78212
78213
78214
78215
78216
78217
78218
78219
78220
78221
78222
78223
78224
78225
78226
78227
78228
78229
78230
78231
78232
78233
78234
78235
78236
78237
78238
78239
78240
78241
78242
78243
78244
78245
78246
78247
78248
78249
78250
78251
78252
78253
78254
78255
78256
78257
78258
78259
78260
78261
78262
78263
78264
78265
78266
78268
78269
78270
78275
78278
78280
78283
78284
78285
78286
78287
78288
78289
78291
78292
78293
78294
78295
78296
78297
78298


In [17]:
# Check the shape of the data frame
print(SA_venues.shape)
SA_venues.head()

(6010, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,78201,29.472,-98.537,Original Donut Shop,29.472703,-98.534598,Donut Shop
1,78201,29.472,-98.537,Restaurant Depot,29.473163,-98.535505,Kitchen Supply Store
2,78201,29.472,-98.537,Pancake Joes,29.464605,-98.543695,Breakfast Spot
3,78201,29.472,-98.537,Tink-A-Tako,29.477054,-98.539218,Mexican Restaurant
4,78201,29.472,-98.537,Jacala Mexican Restaurant,29.468267,-98.525847,Mexican Restaurant


In [18]:
# See how many venues were returned by neighborhood
SA_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
78201,100,100,100,100,100,100
78202,100,100,100,100,100,100
78203,71,71,71,71,71,71
78204,100,100,100,100,100,100
78205,100,100,100,100,100,100
78206,100,100,100,100,100,100
78207,72,72,72,72,72,72
78208,100,100,100,100,100,100
78209,100,100,100,100,100,100


In [19]:
# Find out how many unique categories there are from the returned venues
print('There are {} uniques categories.'.format(len(SA_venues['Venue Category'].unique())))

There are 302 uniques categories.


In [20]:
# Analyze each of the zip codes

# one hot encoding
SA_onehot = pd.get_dummies(SA_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
SA_onehot['Neighborhood'] = SA_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [SA_onehot.columns[-1]] + list(SA_onehot.columns[:-1])
SA_onehot = SA_onehot[fixed_columns]

SA_onehot.head()

,Zoo Exhibit,Accessories Store,Adult Boutique,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Caribbean Restaurant,Casino,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,College Football Field,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Credit Union,Cupcake Shop,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Housing Development,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Office,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Pool Hall,Private School,Pub,Public Art,RV Park,Real Estate Office,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Storage Facility,Strip Club,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tanning Salon,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vin

In [21]:
# Examine the dataframe size
SA_onehot.shape

(6010, 302)

In [22]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
SA_grouped = SA_onehot.groupby('Neighborhood').mean().reset_index()
SA_grouped

,Neighborhood,Zoo Exhibit,Accessories Store,Adult Boutique,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Caribbean Restaurant,Casino,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,College Football Field,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Credit Union,Cupcake Shop,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Housing Development,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Office,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Pool Hall,Private School,Pub,Public Art,RV Park,Real Estate Office,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Storage Facility,Strip Club,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tanning Salon,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vin

In [23]:
# Confirm the new size
SA_grouped.shape

(87, 302)

In [24]:
# Print each zip code with the top 5 venues

num_top_venues = 5

for hood in SA_grouped['Neighborhood']:
    print("----", hood, "----")
    temp = SA_grouped[SA_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- 78201 ----
                  venue  freq
0    Mexican Restaurant  0.14
1        Discount Store  0.08
2  Fast Food Restaurant  0.07
3     Convenience Store  0.05
4          Burger Joint  0.04


---- 78202 ----
            venue  freq
0           Hotel  0.13
1       BBQ Joint  0.04
2  Sandwich Place  0.04
3      Steakhouse  0.03
4          Museum  0.03


---- 78203 ----
                venue  freq
0               Hotel  0.13
1  Mexican Restaurant  0.07
2         Coffee Shop  0.06
3          Steakhouse  0.04
4        Burger Joint  0.04


---- 78204 ----
                venue  freq
0  Mexican Restaurant  0.13
1         Gas Station  0.05
2           BBQ Joint  0.04
3  Seafood Restaurant  0.04
4             Brewery  0.03


---- 78205 ----
            venue  freq
0           Hotel  0.21
1             Bar  0.05
2      Steakhouse  0.04
3         Theater  0.04
4  Ice Cream Shop  0.03


---- 78206 ----
                 venue  freq
0                Hotel  0.15
1   Mexican Restaurant  0.05
2  

In [25]:
# Now we put the venues in a pandas dataframe

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
# Create a new dataframe with the top 10 venues for each zip code

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = SA_grouped['Neighborhood']

for ind in np.arange(SA_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(SA_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,78201,Mexican Restaurant,Discount Store,Fast Food Restaurant,Convenience Store,Pizza Place,Coffee Shop,Burger Joint,Sandwich Place,Grocery Store,Gym / Fitness Center
1,78202,Hotel,BBQ Joint,Sandwich Place,Ice Cream Shop,Cocktail Bar,Steakhouse,Museum,Theater,Sports Bar,Brewery
2,78203,Hotel,Mexican Restaurant,Coffee Shop,BBQ Joint,Steakhouse,Burger Joint,Sandwich Place,Gym / Fitness Center,Sports Bar,Football Stadium
3,78204,Mexican Restaurant,Gas Station,Seafood Restaurant,BBQ Joint,Discount Store,Park,Beer Garden,Trail,Brewery,Burger Joint
4,78205,Hotel,Bar,Theater,Steakhouse,Plaza,Sandwich Place,Ice Cream Shop,American Restaurant,Park,Bakery


In [27]:
# Now let's cluster the neighborhoods

# set number of clusters
kclusters = 5

SA_grouped_clustering = SA_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(SA_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 0, 3, 0, 0, 3, 1, 1, 3], dtype=int32)

In [28]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

SA_merged = neighborhoods

# merge SA_grouped with SA_data to add latitude/longitude for each neighborhood
SA_merged = SA_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Zipcode')

SA_merged.head() # check the last columns!

,Zipcode,City,State,AreaCode,County,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,78201,San Antonio,TX,210,Bexar,29.472,-98.537,3,Mexican Restaurant,Discount Store,Fast Food Restaurant,Convenience Store,Pizza Place,Coffee Shop,Burger Joint,Sandwich Place,Grocery Store,Gym / Fitness Center
1,78202,San Antonio,TX,210,Bexar,29.422,-98.466,0,Hotel,BBQ Joint,Sandwich Place,Ice Cream Shop,Cocktail Bar,Steakhouse,Museum,Theater,Sports Bar,Brewery
2,78203,San Antonio,TX,210,Bexar,29.415,-98.462,0,Hotel,Mexican Restaurant,Coffee Shop,BBQ Joint,Steakhouse,Burger Joint,Sandwich Place,Gym / Fitness Center,Sports Bar,Football Stadium
3,78204,San Antonio,TX,210,Bexar,29.397,-98.500,3,Mexican Restaurant,Gas Station,Seafood Restaurant,BBQ Joint,Discount Store,Park,Beer Garden,Trail,Brewery,Burger Joint
4,78205,San Antonio,TX,210,Bexar,29.424,-98.487,0,Hotel,Bar,Theater,Steakhouse,Plaza,Sandwich Place,Ice Cream Shop,American Restaurant,Park,Bakery


In [35]:
# create map of the clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(SA_merged['Latitude'], SA_merged['Longitude'], SA_merged['Zipcode'], SA_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

Lets examine the clusters

In [43]:
# Cluster 0
SA_merged.loc[SA_merged['Cluster Labels'] == 0, SA_merged.columns[[1] + list(range(5, SA_merged.shape[1]))]]

,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,San Antonio,29.422,-98.466,0,Hotel,BBQ Joint,Sandwich Place,Ice Cream Shop,Cocktail Bar,Steakhouse,Museum,Theater,Sports Bar,Brewery
2,San Antonio,29.415,-98.462,0,Hotel,Mexican Restaurant,Coffee Shop,BBQ Joint,Steakhouse,Burger Joint,Sandwich Place,Gym / Fitness Center,Sports Bar,Football Stadium
4,San Antonio,29.424,-98.487,0,Hotel,Bar,Theater,Steakhouse,Plaza,Sandwich Place,Ice Cream Shop,American Restaurant,Park,Bakery
5,San Antonio,29.417,-98.499,0,Hotel,Mexican Restaurant,Plaza,Theater,Park,Steakhouse,American Restaurant,Cocktail Bar,Beer Garden,BBQ Joint
14,San Antonio,29.438,-98.481,0,Hotel,Bar,Burger Joint,Brewery,Cocktail Bar,Museum,Sandwich Place,Restaurant,Coffee Shop,Theater
72,San Antonio,29.411,-98.504,0,Hotel,Mexican Restaurant,Theater,Plaza,Park,BBQ Joint,American Restaurant,Beer Garden,Cocktail Bar,Lounge
77,San Antonio,29.426,-98.492,0,Hotel,Mexican Restaurant,Ice Cream Shop,Theater,Steakhouse,Plaza,American Restaurant,Cocktail Bar,Sandwich Place,Seafood Restaurant
79,San Antonio,29.426,-98.488,0,Hotel,Bar,Steakhouse,Theater,Ice Cream Shop,American Restaurant,Cocktail Bar,Sandwich Place,Seafood Restaurant,Plaza
80,San Antonio,29.426,-98.486,0,Hotel,Bar,Theater,Steakhouse,Cocktail Bar,Plaza,Sandwich Place,Ice Cream Shop,American Restaurant,Bakery
81,San Antonio,29.425,-98.487,0,Hotel,Bar,Steakhouse,Theater,Ice Cream Shop,American Restaurant,Cocktail Bar,Sandwich Place,Plaza,Dessert Shop


In [44]:
# Cluster 1
SA_merged.loc[SA_merged['Cluster Labels'] == 1, SA_merged.columns[[1] + list(range(5, SA_merged.shape[1]))]]

,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,San Antonio,29.438,-98.457,1,Coffee Shop,Sandwich Place,Mexican Restaurant,Burger Joint,Southern / Soul Food Restaurant,Cocktail Bar,Restaurant,New American Restaurant,Beer Garden,American Restaurant
8,San Antonio,29.488,-98.457,1,Coffee Shop,Pizza Place,Bakery,Grocery Store,Italian Restaurant,Mexican Restaurant,Ice Cream Shop,American Restaurant,Sporting Goods Shop,Frozen Yogurt Shop
11,San Antonio,29.462,-98.494,1,Mexican Restaurant,Bar,Burger Joint,Park,Coffee Shop,American Restaurant,Gay Bar,Pizza Place,Flower Shop,Thrift / Vintage Store
12,San Antonio,29.512,-98.524,1,Mexican Restaurant,Burger Joint,Thai Restaurant,Pizza Place,Pharmacy,Convenience Store,Fast Food Restaurant,Sushi Restaurant,Asian Restaurant,Music Store
15,San Antonio,29.534,-98.489,1,Hotel,Mexican Restaurant,Clothing Store,American Restaurant,Cosmetics Shop,Department Store,Toy / Game Store,Seafood Restaurant,Pizza Place,Snack Place
28,San Antonio,29.501,-98.574,1,Mexican Restaurant,Coffee Shop,Sandwich Place,Chinese Restaurant,Pharmacy,Bar,Bakery,Fast Food Restaurant,Indian Restaurant,Fried Chicken Joint
29,San Antonio,29.540,-98.552,1,Mexican Restaurant,Coffee Shop,Burger Joint,Grocery Store,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Bar,Mediterranean Restaurant,Seafood Restaurant
30,San Antonio,29.577,-98.539,1,Pizza Place,Video Store,Pharmacy,Gas Station,Gym / Fitness Center,Cosmetics Shop,Convenience Store,Fast Food Restaurant,BBQ Joint,Trail
31,San Antonio,29.585,-98.475,1,Mexican Restaurant,Convenience Store,Burger Joint,Coffee Shop,Bar,Italian Restaurant,Fast Food Restaurant,Taco Place,Pizza Place,Ice Cream Shop
33,San Antonio,29.461,-98.439,1,Sandwich Place,Hotel,Shipping Store,Department Store,BBQ Joint,Coffee Shop,American Restaurant,Spa,Bagel Shop,Gym


In [45]:
# Cluster 2
SA_merged.loc[SA_merged['Cluster Labels'] ==2, SA_merged.columns[[1] + list(range(5, SA_merged.shape[1]))]]

,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,San Antonio,29.193,-98.517,2,Restaurant,Food,Massage Studio,Zoo,Flower Shop,Fondue Restaurant,Food & Drink Shop,Food Court,Food Service,Food Truck


In [46]:
# Cluster 3
SA_merged.loc[SA_merged['Cluster Labels'] == 3, SA_merged.columns[[1] + list(range(5, SA_merged.shape[1]))]]

,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,San Antonio,29.472,-98.537,3,Mexican Restaurant,Discount Store,Fast Food Restaurant,Convenience Store,Pizza Place,Coffee Shop,Burger Joint,Sandwich Place,Grocery Store,Gym / Fitness Center
3,San Antonio,29.397,-98.500,3,Mexican Restaurant,Gas Station,Seafood Restaurant,BBQ Joint,Discount Store,Park,Beer Garden,Trail,Brewery,Burger Joint
6,San Antonio,29.422,-98.523,3,Mexican Restaurant,Hotel,Discount Store,Pizza Place,Fast Food Restaurant,Gas Station,Fried Chicken Joint,Chinese Restaurant,Bar,Thrift / Vintage Store
9,San Antonio,29.399,-98.470,3,Mexican Restaurant,Discount Store,Fast Food Restaurant,Bakery,BBQ Joint,Burger Joint,Convenience Store,Fried Chicken Joint,Bar,Southern / Soul Food Restaurant
10,San Antonio,29.357,-98.560,3,Fast Food Restaurant,Mexican Restaurant,Discount Store,Sandwich Place,Fried Chicken Joint,Gym / Fitness Center,Hardware Store,Big Box Store,Burger Joint,Breakfast Spot
13,San Antonio,29.366,-98.487,3,Mexican Restaurant,Fast Food Restaurant,Convenience Store,Fried Chicken Joint,Pizza Place,Bar,Burger Joint,Seafood Restaurant,Chinese Restaurant,Sandwich Place
16,San Antonio,29.544,-98.420,3,Mexican Restaurant,Discount Store,Bar,Sandwich Place,Food Truck,Baseball Field,Latin American Restaurant,Golf Course,Taco Place,Sports Bar
17,San Antonio,29.501,-98.398,3,Mexican Restaurant,American Restaurant,Gas Station,BBQ Joint,Discount Store,Gym / Fitness Center,Wings Joint,Sandwich Place,Burger Joint,Hotel
18,San Antonio,29.440,-98.395,3,Gas Station,Discount Store,Convenience Store,Mexican Restaurant,Hotel,Seafood Restaurant,Sandwich Place,Pizza Place,Rental Car Location,Fried Chicken Joint
19,San Antonio,29.411,-98.404,3,Convenience Store,Fast Food Restaurant,Discount Store,BBQ Joint,Pizza Place,Park,Hotel,Fried Chicken Joint,Mexican Restaurant,Clothing Store


In [47]:
# Cluster 4
SA_merged.loc[SA_merged['Cluster Labels'] == 4, SA_merged.columns[[1] + list(range(5, SA_merged.shape[1]))]]

,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
60,San Antonio,29.706,-98.399,4,Hunting Supply,Brewery,Construction & Landscaping,Home Service,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain,Furniture / Home Store
